# Python & SQLite – Teil 2

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Verbindung zur Datenbank herstellen
database_path = r'C:\Users\Admin\OneDrive\Dokumente\DataCraft\DataAnalyst_Aug.24\08_Datenbanken_und_SQL\databases\wohnungen.db'
connection = sqlite3.connect(database_path)

In [3]:
# Cursor erstellen
cursor = connection.cursor()

In [4]:
# Verbindung testen
cursor.execute('SELECT * FROM hotels;')

In [5]:
# Das Ergebnis von fetchall() ist eine Liste
result = cursor.fetchall()

In [6]:
result

[(119000, 21.88, 3938, 'Berlin', 5556.119857795836),
 (250000, 27.95, 3986, 'München', 7012.042147516308),
 (250000, 16.09, 2574, 'Köln', 6250.971250971251),
 (145000, 27.58, 4155, 'München', 6637.785800240674),
 (110000, 23.76, 3795, 'Berlin', 6260.869565217391),
 (246000, 22.88, 2773, 'München', 8250.991705733863),
 (54000, 13.25, 634, 'München', 20899.05362776025),
 (2000, 8.94, 82, 'München', 109024.3902439024),
 (114000, 24.87, 3706, 'München', 6710.739341608203),
 (47000, 14.11, 1692, 'Berlin', 8339.243498817967),
 (54000, 11.65, 1989, 'Köln', 5857.214680744092),
 (124000, 17.26, 2616, 'Berlin', 6597.859327217126),
 (125000, 18.45, 3358, 'Köln', 5494.34187016081),
 (62000, 17.08, 1941, 'München', 8799.587841318908),
 (250000, 19.32, 1831, 'München', 10551.61114145276),
 (24000, 8.48, 800, 'Berlin', 10600.0),
 (102000, 14.16, 2700, 'Köln', 5244.444444444444),
 (26000, 13.84, 1257, 'München', 11010.34208432777),
 (85000, 19.59, 2644, 'Berlin', 7409.228441754917),
 (182000, 18.64, 2

In [7]:
# Python-Wiederholung:
# Wie greifen wir auf das dritte Element der Liste zu?
result[2]

(250000, 16.09, 2574, 'Köln', 6250.971250971251)

In [8]:
# Python-Wiederholung:
# Ein einzelnes Element des Ergebnisses von fetchall()
# ist ein Tupel. Wie greifen wir auf das vierte Element
# des dritten Tupels zu?
result[2][3]

'Köln'

In [9]:
# Warum haben wir fetchall auf Variable result gelegt?
# weil wenn wir es nicht auf die var packen, ist der cursor nach dem 1. fetchall() leer und kann nicht mehr verwendet werden!

## SQL-Abfrage-Ergebnisse direkt in DataFrames speichern

Daten aus Datenbank auslesen und in DataFrame speichern
-> Für die Umwandlung nutzen wir `pd.read_sql`

In [10]:
help(pd.read_sql)

Help on function read_sql in module pandas.io.sql:

read_sql(sql, con, index_col: 'str | list[str] | None' = None, coerce_float: 'bool' = True, params=None, parse_dates=None, columns: 'list[str] | None' = None, chunksize: 'int | None' = None, dtype_backend: 'DtypeBackend | lib.NoDefault' = <no_default>, dtype: 'DtypeArg | None' = None) -> 'DataFrame | Iterator[DataFrame]'
    Read SQL query or database table into a DataFrame.

    This function is a convenience wrapper around ``read_sql_table`` and
    ``read_sql_query`` (for backward compatibility). It will delegate
    to the specific function depending on the provided input. A SQL query
    will be routed to ``read_sql_query``, while a database table name will
    be routed to ``read_sql_table``. Note that the delegated function might
    have more specific notes about their functionality not listed here.

    Parameters
    ----------
    sql : str or SQLAlchemy Selectable (select or text object)
        SQL query to be executed or

In [9]:
hotels_df = pd.read_sql('SELECT * FROM hotels;', connection)
hotels_df.head()

,Gewinn,Preis in Mio,Quadratmeter,Stadt,Preis pro Quadratmeter
0,119000,21.88,3938,Berlin,5556.119858
1,250000,27.95,3986,München,7012.042148
2,250000,16.09,2574,Köln,6250.971251
3,145000,27.58,4155,München,6637.785800
4,110000,23.76,3795,Berlin,6260.869565


#### Das Problem mit den Leerzeichen

In Python sollte man Leerzeichen (außerhalb von Strings) am besten vermeiden.
Allerdings sollte man auch innerhalb von Datenbanken auf Leerzeichen
verzichten, weil sie uns spätestens in Python wieder Probleme bereiten.

In [10]:
# Alle Infos der Spalte "Preis in Mio" selektieren:
df_preis = pd.read_sql('SELECT Preis in Mio FROM hotels;', connection)

DatabaseError: Execution failed on sql 'SELECT Preis in Mio FROM hotels;': no such table: Mio

In [11]:
# Der vorherige Code verursacht einen Fehler, da die Bezeichnung
# "Preis in Mio" nicht als zusammengehöriger String identifiziert wird
# Wie lösen wir das?
df_preis = pd.read_sql('SELECT "Preis in Mio" FROM hotels;', connection)
df_preis

,Preis in Mio
0,21.88
1,27.95
2,16.09
3,27.58
4,23.76
...,...
146,23.80
147,12.86
148,15.80
149,17.09


In [12]:
connection.close()

### 2. Einmalige SQL-Verbindung
#### 2.1 `with`-Statement 

 Das `with`-Statement haben wir bereits im Zusammenhang mit dem Öffnen, Lesen und Schreiben von Dateien kennengelernt. Praktisch daran war, dass es eine Datei geöffnet und anschließend automatisch geschlossen hat. Den gleichen Vorteil können wir mit sqlite3 mit Datenbanken leider nicht nutzen.

In [13]:
with sqlite3.connect(database_path) as connection:
	df = pd.read_sql('SELECT * FROM hotels;', connection)

df.head()

,Gewinn,Preis in Mio,Quadratmeter,Stadt,Preis pro Quadratmeter
0,119000,21.88,3938,Berlin,5556.119858
1,250000,27.95,3986,München,7012.042148
2,250000,16.09,2574,Köln,6250.971251
3,145000,27.58,4155,München,6637.785800
4,110000,23.76,3795,Berlin,6260.869565


In [14]:
# Trotz abgeschlossenem with Block, kann trotzdem auf Verbindung
# zugegriffen werden
connection.cursor() \
	.execute('SELECT * FROM hotels;') \
	.fetchall()

[(119000, 21.88, 3938, 'Berlin', 5556.119857795836),
 (250000, 27.95, 3986, 'München', 7012.042147516308),
 (250000, 16.09, 2574, 'Köln', 6250.971250971251),
 (145000, 27.58, 4155, 'München', 6637.785800240674),
 (110000, 23.76, 3795, 'Berlin', 6260.869565217391),
 (246000, 22.88, 2773, 'München', 8250.991705733863),
 (54000, 13.25, 634, 'München', 20899.05362776025),
 (2000, 8.94, 82, 'München', 109024.3902439024),
 (114000, 24.87, 3706, 'München', 6710.739341608203),
 (47000, 14.11, 1692, 'Berlin', 8339.243498817967),
 (54000, 11.65, 1989, 'Köln', 5857.214680744092),
 (124000, 17.26, 2616, 'Berlin', 6597.859327217126),
 (125000, 18.45, 3358, 'Köln', 5494.34187016081),
 (62000, 17.08, 1941, 'München', 8799.587841318908),
 (250000, 19.32, 1831, 'München', 10551.61114145276),
 (24000, 8.48, 800, 'Berlin', 10600.0),
 (102000, 14.16, 2700, 'Köln', 5244.444444444444),
 (26000, 13.84, 1257, 'München', 11010.34208432777),
 (85000, 19.59, 2644, 'Berlin', 7409.228441754917),
 (182000, 18.64, 2

In [17]:
# Manuelles schließen
connection.close()

ABER ganz nutzlos ist with mit sqlite3 auch nicht:
Wir können auf connection.commit() verzichten durch 'with'

In [15]:
with sqlite3.connect(database_path) as connection:
	connection.execute('''INSERT INTO hotels
                          VALUES (12000, 15.5, 222, 'Zutzenhausen', 12.2)''')

In [16]:
with sqlite3.connect(database_path) as connection:
	df = pd.read_sql("SELECT * FROM hotels;", connection)

In [17]:
df.tail()

,Gewinn,Preis in Mio,Quadratmeter,Stadt,Preis pro Quadratmeter
147,32000,12.86,1668,Berlin,7709.832134
148,35000,15.80,2281,Berlin,6926.786497
149,90000,17.09,2297,Berlin,7440.139312
150,12000,15.50,222,Zutzenhausen,12.200000
151,12000,15.50,222,Zutzenhausen,12.200000


In [22]:
connection.close()

#### 2.2 `try` - Statement

In [18]:
# Alternativ dazu können wir einen try-except-finally-
# Block definieren
try:
	connection = sqlite3.connect(database_path)
	cursor = connection.cursor()
	cursor.execute("SELECT * FROM hotels")
	print(cursor.fetchall())
except:
	print('Irgendwas hat da nicht geklappt.')
finally:
	connection.close()

[(119000, 21.88, 3938, 'Berlin', 5556.119857795836), (250000, 27.95, 3986, 'München', 7012.042147516308), (250000, 16.09, 2574, 'Köln', 6250.971250971251), (145000, 27.58, 4155, 'München', 6637.785800240674), (110000, 23.76, 3795, 'Berlin', 6260.869565217391), (246000, 22.88, 2773, 'München', 8250.991705733863), (54000, 13.25, 634, 'München', 20899.05362776025), (2000, 8.94, 82, 'München', 109024.3902439024), (114000, 24.87, 3706, 'München', 6710.739341608203), (47000, 14.11, 1692, 'Berlin', 8339.243498817967), (54000, 11.65, 1989, 'Köln', 5857.214680744092), (124000, 17.26, 2616, 'Berlin', 6597.859327217126), (125000, 18.45, 3358, 'Köln', 5494.34187016081), (62000, 17.08, 1941, 'München', 8799.587841318908), (250000, 19.32, 1831, 'München', 10551.61114145276), (24000, 8.48, 800, 'Berlin', 10600.0), (102000, 14.16, 2700, 'Köln', 5244.444444444444), (26000, 13.84, 1257, 'München', 11010.34208432777), (85000, 19.59, 2644, 'Berlin', 7409.228441754917), (182000, 18.64, 2001, 'München', 931

## Prepared statement in SQL
"Prepared statements" verbessern zum einen die Laufzeit von Queries, welche mehrmals mit unterschiedlichen Werten aufgerufen werden. Außerdem bieten sie einen wichtigen Schutz gegen Angriffe von außen durch sogenannte SQL-Injections (Einschleusung von SQL-Code). 

### 1. Question Mark Style
1.1 Insert mit Question Mark style. Hier stehen '?' stellvertretend für beliebige Werte

In [44]:
connection = sqlite3.connect(database_path)
cursor = connection.cursor()

In [45]:
# Tabelle erstellen:
connection.execute(
	'''CREATE TABLE IF NOT EXISTS programmiersprachen(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(50) UNIQUE NOT NULL,
            first_appeared INT);'''
)

In [46]:
# Fragezeichen können als Platzhalter dienen, um dann einen Befehl
# mehrmals mit unterschiedlichen Werten ausführen zu können

# Die einzutragenden Werte, jedes Tupel ist ein Eintrag:
data = [
	("Ada Lovelace Machinealgorithm", 1843),
	("Assembler", 1949),
	("COBOL", 1959),
	("SQL", 1972),
	("C++", 1985),
	("Python", 1991),
	("Swift", 2014)
]

In [47]:
# Mit executemany kann man einen Befehl wiederholt auf eine iterierbare Datenstruktur anwenden.
# In unserem Fall liegt eine Liste aus Tupeln vor und executemany führt das Insert so oft aus,
# wie es Tupel in der Liste gibt.

# Da die Tupel jeweils über zwei Werte verfügen, werden sie von VALUES(?, ?) abgebildet:
cursor.executemany('''
                  INSERT INTO programmiersprachen(name, first_appeared)
                  VALUES (?, ?)''', data)

In [30]:
# Was ist eigentlich mit der ID-Spalte?
# ...AUTOINCREMENT

[]

In [23]:
help(cursor.executemany)

Help on built-in function executemany:

executemany(sql, seq_of_parameters, /) method of sqlite3.Cursor instance
    Repeatedly executes an SQL statement.



In [48]:
connection.commit()

In [42]:
cursor.execute('SELECT * FROM programmiersprachen;').fetchall()

[(17, 'Ada Lovelace Machinealgorithm', 1843),
 (18, 'Assembler', 1949),
 (19, 'COBOL', 1959),
 (20, 'SQL', 1972),
 (21, 'C++', 1985),
 (22, 'Python', 1991),
 (23, 'Swift', 2014)]

1.2 Tabelle ausgeben lassen

In [26]:
result = cursor.execute('SELECT * FROM programmiersprachen;').fetchall()
for row in result:
	print(row)

(1, 'Ada Lovelace Machinealgorithm', 1843)
(2, 'Assembler', 1949)
(3, 'COBOL', 1959)
(4, 'SQL', 1972)
(5, 'C++', 1985)
(6, 'Python', 1991)
(7, 'Swift', 2014)
(8, 'Ada Lovelace Machinealgorithm', 1843)
(9, 'Assembler', 1949)
(10, 'COBOL', 1959)
(11, 'SQL', 1972)
(12, 'C++', 1985)
(13, 'Python', 1991)
(14, 'Swift', 2014)


In [27]:
# Tuple unpacking:
result = cursor.execute('SELECT * FROM programmiersprachen;').fetchall()
for _, name, year in result:
	print('Name der Sprache:', name)
	print('Erscheinungsjahr:', year)
	print()

Name der Sprache: Ada Lovelace Machinealgorithm
Erscheinungsjahr: 1843

Name der Sprache: Assembler
Erscheinungsjahr: 1949

Name der Sprache: COBOL
Erscheinungsjahr: 1959

Name der Sprache: SQL
Erscheinungsjahr: 1972

Name der Sprache: C++
Erscheinungsjahr: 1985

Name der Sprache: Python
Erscheinungsjahr: 1991

Name der Sprache: Swift
Erscheinungsjahr: 2014

Name der Sprache: Ada Lovelace Machinealgorithm
Erscheinungsjahr: 1843

Name der Sprache: Assembler
Erscheinungsjahr: 1949

Name der Sprache: COBOL
Erscheinungsjahr: 1959

Name der Sprache: SQL
Erscheinungsjahr: 1972

Name der Sprache: C++
Erscheinungsjahr: 1985

Name der Sprache: Python
Erscheinungsjahr: 1991

Name der Sprache: Swift
Erscheinungsjahr: 2014



1.3 Select mit Question Mark style

In [28]:
abfrage = 'SELECT * FROM programmiersprachen WHERE name = ?;'
# (Python,) steht für ein Tupel mit nur EINEM Element:
cursor.execute(abfrage, ('Python',))
cursor.fetchall()

[(6, 'Python', 1991), (13, 'Python', 1991)]

In [29]:
abfrage = 'SELECT * FROM programmiersprachen WHERE name = ? OR first_appeared = ?;'
# (Python,) steht für ein Tupel mit nur EINEM Element:
cursor.execute(abfrage, ('Python', 1843))
cursor.fetchall()

[(1, 'Ada Lovelace Machinealgorithm', 1843),
 (6, 'Python', 1991),
 (8, 'Ada Lovelace Machinealgorithm', 1843),
 (13, 'Python', 1991)]

1.4 Verwendung in Python-Funktionen

In [30]:
# Funktion erstellen, mit der wir neue Zeilen
# in Tabelle 'hotels' eintragen können:
def add_hotel(gewinn, preis, qm, stadt, qm_preis):
	query = '''INSERT INTO hotels
               VALUES(?, ?, ?, ?, ?);'''
	cursor.execute(query, (gewinn, preis, qm, stadt, qm_preis))
	connection.commit()
	return 'Hotel wurde hinzugefügt!'

In [31]:
# Funktion aufrufen:
add_hotel(125641, 15, 300, "Bamberg", 3500)

'Hotel wurde hinzugefügt!'

In [49]:
# Zu programmiersprachen mit with-Schreibweise hinzufügen.
# Funktion (durch with kein commit mehr nötig!):
def add_language(name, year):
	query = '''INSERT INTO programmiersprachen(name, first_appeared)
               VALUES(?, ?);'''
	with connection:
		connection.execute(query, (name, year))
	return 'Programmiersprache hinzugefügt!'

In [50]:
add_language('Rust', 2006)

'Programmiersprache hinzugefügt!'

In [ ]:
# Eventuelle Übungsaufgabe:
# Eine Funktion schreiben, die zu areas neue Zeilen hinzufügt.

In [51]:
def add_row(preis, quadratmeter, stadt):
	query = '''
	INSERT INTO areas(preis, quadratmeter, stadt)
	VALUES (?, ?, ?);
	'''
	with connection:
		connection.execute(query, (preis, quadratmeter, stadt))
		return 'Stadt wurde hinzugefügt!'

In [52]:
add_row(133.42, 75, 'Hörnum/Sylt')

'Stadt wurde hinzugefügt!'

### 2. Named style
2.1 Insert mit named style

In [53]:
# Mithilfe eines Dictionaries können Keys der Zuordnung der Werte dienen
# Dadurch müssen Werte nicht in richtiger Reihenfolge stehen:
name_sql = """
            INSERT INTO programmiersprachen(name, first_appeared)
            VALUES(:name, :first_appeared);"""

sprache = {'first_appeared': 1993, 'name': 'Brainfuck'}

connection.execute(name_sql, sprache)

In [54]:
connection.commit()

In [55]:
cursor.execute("SELECT * FROM programmiersprachen")
cursor.fetchall()

[(1, 'Ada Lovelace Machinealgorithm', 1843),
 (2, 'Assembler', 1949),
 (3, 'COBOL', 1959),
 (4, 'SQL', 1972),
 (5, 'C++', 1985),
 (6, 'Python', 1991),
 (7, 'Swift', 2014),
 (8, 'Rust', 2006),
 (9, 'Brainfuck', 1993)]

2.2 Tabelle mit where-Bedingung ausgeben (named style)

In [56]:
cursor.execute("""
               SELECT *
               FROM programmiersprachen
               WHERE first_appeared > :year;""",
			   {"year": 1990})

cursor.fetchall()

[(6, 'Python', 1991),
 (7, 'Swift', 2014),
 (8, 'Rust', 2006),
 (9, 'Brainfuck', 1993)]

In [57]:
# Schließen nicht vergessen!
connection.close()

### 3. Warum nicht einfach f-String? >>> SQL-Injections!
Lasst uns eine ganze Tabelle voller "geheimer Inhalte" klauen!

In [58]:
connection = sqlite3.connect(database_path)

In [59]:
with connection:
	connection.execute('''CREATE TABLE IF NOT EXISTS super_confident(
                               password VARCHAR,
                               secret_content VARCHAR);
                               ''')

In [60]:
data = [('21412452d', 'Daten zum Konto in der Schweiz'),
		('Zdsam832197m', 'Das größte Geheimnis'),
		('998321_dsHwoepw§', 'Die Weltformel')]

with connection:
	connection.executemany('''INSERT INTO super_confident
                              VALUES(?, ?)''', data)

In [61]:
cursor = connection.cursor()
cursor.execute('SELECT * FROM super_confident;')
cursor.fetchall()

[('21412452d', 'Daten zum Konto in der Schweiz'),
 ('Zdsam832197m', 'Das größte Geheimnis'),
 ('998321_dsHwoepw§', 'Die Weltformel')]

In [62]:
# So sollten Nutzer auf ihre Geheimnisse zugreifen:
code = 'Zdsam832197m'
cursor.execute(f'''SELECT * FROM super_confident WHERE password = '{code}';''')

In [63]:
cursor.fetchall()

[('Zdsam832197m', 'Das größte Geheimnis')]

In [64]:
# Jetzt kommt der "Angriff":
injection = '1 OR 1=1'
cursor.execute(f'''SELECT * FROM super_confident
                   WHERE password = {injection}''')

In [65]:
# Die Katastrophe: Wir haben nun die komplette Tabelle!
cursor.fetchall()

[('21412452d', 'Daten zum Konto in der Schweiz'),
 ('Zdsam832197m', 'Das größte Geheimnis'),
 ('998321_dsHwoepw§', 'Die Weltformel')]

In [66]:
# Was passiert, wenn wir '?' schreiben?
injection = '1 OR 1=1'
cursor.execute('''SELECT * FROM super_confident
                   WHERE password = ?''', (injection,))

In [67]:
# So sieht es schon besser aus!
cursor.fetchall()

[]

In [ ]:
# Übungsaufgabe: Schreibe weiter an app.py und database.py von gestern.
# Schreibe die Funktion insert_topic so, dass der Nutzer Datum und Lerninhalt eintragen kann,
# aber die Gefahr von SQL-Einschleusung nicht besteht!
# Schreibe die Funktion view_entries aus.
# Falls die Zeit reicht: Verbaue die Funktionen in app.py

In [ ]:
# Berücksichtige den Walrus-Operator:
# https://towardsdatascience.com/the-walrus-operator-in-python-a315e4f84583